<a id="Menu"></a>
<h2>   
      <font color = #66BEAF >
            Table of Contents:   
        </font>    
</h2>
    <ol>
        <li><a href='#T0'>Introduction</a></li>
        <li><a href='#T1'>Import librairies & functions definitions</a></li>
        <li><a href='#T2'>Import data</a></li>
        <li><a href='#T3'>Data overview</a></li>
        <li><a href='#T4'>Data cleaning</a></li>
            <ol>
                <li><a href='#T41'>Missing values overview</a></li>
                <li><a href='#T42'>Removing of columns</a></li>
                <li><a href='#T43'>Recoding fields</a></li>
                <li><a href='#T44'>Outliers management</a></li>
                <li><a href='#T45'>New field</a></li>
            </ol>
        <li><a href='#T5'>Target data overview</a></li>
        <li><a href='#T6'>Fields gruping</a></li>
        <li><a href='#T7'>Fields analysis by subject</a></li>
            <ol>
                <li><a href='#T71'>Family</a></li>
                <li><a href='#T72'>Location</a></li>
                <li><a href='#T73'>Revenues / Job</a></li>
                <li><a href='#T74'>Contact variables</a></li>
                <li><a href='#T75'>Loan variables</a></li>
                <li><a href='#T76'>Other</a></li>
            </ol>
        <li><a href='#T8'>Final fields selection</a></li>
        <li><a href='#T9'>Data preparation for modeling</a></li>
                <ol>
                <li><a href='#T91'>Database centered reduced</a></li>
                <li><a href='#T92'>Train/Validation tables</a></li>
                </ol>
        <li><a href='#T10'>Score construction</a></li>
            <ol>
                <li><a href='#T101'>Logistic regression</a></li>
                <li><a href='#T102'>Performance</a></li>
                <li><a href='#T103'>Score grid</a></li>
                <li><a href='#T104'>Cutt offs</a></li>
                <li><a href='#T105'>Score calculation on the whole database</a></li>
            </ol>
        <li><a href='#T11'>Conclusion</a>

<a id="T0"></a>
<h2>   
      <font size="6" , color = #26B260>
            1. Introduction
        </font> 
</h2>

Objective of this notebook is to build a credit scorcard based on "application data" from the step of importing data to the calculation of the scores on the whole database. The modeling method choosen is logistic regression.<br><br>
Technics used in this analysis : 
- Homogeneity tests of proportions
- Grouping of modalities by decision tree
- WoE and IV calculation
- Correlation analysis
- Model performance measurement (AUC / ROC)
- Scoring Grid
- Cut offs

This notebook is composed of following parts : importing data, general overview of data, data cleaning, selection of fields, modelization and score calculation. 

All remarks on methodology or python programmation improvement are welcomes.

<a id="T1"></a>
<h2>   
      <font size="6" , color = #26B260>
            2. Import librairies & functions definitions:
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

Librairies : 

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 

pd.options.mode.chained_assignment = None 

import warnings
warnings.filterwarnings("ignore")

Function to manage outliers : 

In [ ]:
def GraphOutliers(col,titl,zoom) : 
    plt.figure(figsize=(15,8))
    plt.subplot(2,2,1)
    sns.boxplot(y=df_application[col])
    plt.title(titl)
    
    # Imputimg Null values with median to draw distplot
    plt.subplot(2,2,2)
    df_application[col].fillna((df_application[col].median()), inplace=True) 
    sns.distplot(df_application[col])
    
    # After removing outliers
    plt.subplot(2,2,3)
    sns.boxplot(df_application[df_application[col]<zoom][col])
    plt.title(titl)
    plt.show()

Function to manage homogeneity tests of proportions : 

In [ ]:
def TestHomog (col) : 
    aggregations = {'TARGET': [sum,'count'],}
    res=df_application.groupby(col).agg(aggregations).reset_index()
    nobstot = res.iloc[:,-1]
    nobs=res.iloc[:,-2]
    #Test proportions 
    from statsmodels.stats.proportion import proportions_ztest
    stat, pval = proportions_ztest(nobs, nobstot)
    if nobstot[0]<30 or nobstot[1]<30 or nobs[0]/nobstot[0]*nobstot[0]<5 or nobs[1]/nobstot[1]*nobstot[1]<5 or (1-nobs[0]/nobstot[0])*nobstot[0]<5 or (1-nobs[1]/nobstot[1])*nobstot[1]<5 :
        concl = 'At least one condition not verified to do the test' 
    elif pval<0.05 : 
        concl = 'H0 rejection : Proportions are different' 
    else : 
        concl = 'Non H0 rejection : Impossible to determine a difference' 
    print("z=" + str(stat) + " et p=" + str(pval) + "\nConclusion : " + concl)

Function to calculate WOE and IV :

In [ ]:
def calculate_woe_iv(dataset, feature, target):
    lst = []
    for i in range(dataset[feature].nunique()):
        val = dataset[feature].unique().tolist()[i]
        lst.append({
            'Value': val,
            'All': dataset[dataset[feature] == val].count()[feature],
            'Good': dataset[(dataset[feature] == val) & (dataset[target] == 0)].count()[feature],
            'Bad': dataset[(dataset[feature] == val) & (dataset[target] == 1)].count()[feature]
        })
        
    dset = pd.DataFrame(lst)
    dset['Distr_Good'] = dset['Good'] / dset['Good'].sum()
    dset['Distr_Bad'] = dset['Bad'] / dset['Bad'].sum()
    dset['WoE'] = np.log(dset['Distr_Good'] / dset['Distr_Bad'])
    dset = dset.replace({'WoE': {np.inf: 0, -np.inf: 0}})
    dset['IV'] = (dset['Distr_Good'] - dset['Distr_Bad']) * dset['WoE']
    iv = dset['IV'].sum()
    
    dset = dset.sort_values(by='WoE')
    
    return dset, iv

In [ ]:
def IV_Comment (val) : 
    if val<0.02 : 
        return "<0.02 - Not useful for prediction"
    elif val<0.1 : 
        return "<0.1 - Weak predictive power"
    elif val<0.3 : 
        return "<0.3 - Medium predictive power"
    elif val<0.5 : 
        return "<0.5 - Strong predictive power"
    else : 
        return ">=0.5 - Suspicious predictive power"

Function to create disjunctive table :

In [ ]:
def dummy_creation(df, columns_list):
    df_dummies = []
    for col in columns_list:
        df_dummies.append(pd.get_dummies(df[col], prefix = col, prefix_sep = ':'))
    df_dummies = pd.concat(df_dummies, axis = 1)
    df = pd.concat([df, df_dummies], axis = 1)
    return df

<a id="T2"></a>
<h2>   
      <font size="6" , color = #26B260>
            3. Import data
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>


In [ ]:
df_application = pd.read_csv("../input/loan-defaulter/application_data.csv")
df_application.head()

<a id="T3"></a>
<h2>   
      <font size="6" , color = #26B260>
            4. Import overview
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

In [ ]:
df_application.info()

In [ ]:
df_application.columns

In [ ]:
df_application.describe()

<a id="T4"></a>
<h2>   
      <font size="6" , color = #26B260>
            5. Data cleaning
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

<a id="T41"></a>
<h2>   
      <font size="5" , color = #41B6DF>
            5.1. Missing values overview
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

In [ ]:
data = [1] * 100
for i in range(0,99) : 
    data[i]=len(df_application.columns[(100*df_application.isnull().sum()/len(df_application))>=i])

plt.plot(data)
plt.title("Crossing missing values & # columns")
plt.xlabel('% Missing')
plt.ylabel('# Columns')

<a id="T42"></a>
<h2>   
      <font size="5" , color = #41B6DF>
            5.2. Removing of columns
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

Columns names to remove (with more than 40% of missing values) : 

In [ ]:
df_application.columns[(100*df_application.isnull().sum()/len(df_application))>=40]

Removing and checking :

In [ ]:
liste=df_application.columns[(100*df_application.isnull().sum()/len(df_application))>=40]
liste=liste.tolist()
df_application=df_application.drop(liste, axis=1)
df_application.shape

<a id="T43"></a>
<h2>   
      <font size="5" , color = #41B6DF>
            5.3. Recoding fields
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

List of fields to recode due to missing values : 

In [ ]:
df_application.columns[(100*df_application.isnull().sum()/len(df_application))>0]

Recoding by new category :

In [ ]:
df_application["OCCUPATION_TYPE"].loc[df_application["OCCUPATION_TYPE"].isnull()]="NoCom"
df_application["NAME_TYPE_SUITE"].loc[df_application["NAME_TYPE_SUITE"].isnull()]="NoCom"

Recoding by avg :

In [ ]:
df_application.EXT_SOURCE_3[df_application.EXT_SOURCE_3.isna()] = df_application["EXT_SOURCE_3"].mean()
df_application.EXT_SOURCE_2[df_application.EXT_SOURCE_2.isna()] = df_application["EXT_SOURCE_2"].mean()
df_application.AMT_ANNUITY[df_application.AMT_ANNUITY.isna()] = df_application["AMT_ANNUITY"].mean()
df_application.AMT_GOODS_PRICE[df_application.AMT_GOODS_PRICE.isna()] = df_application["AMT_GOODS_PRICE"].mean()

Recoding by 0 :

In [ ]:
df_application.AMT_REQ_CREDIT_BUREAU_HOUR[df_application.AMT_REQ_CREDIT_BUREAU_HOUR.isna()] = 0
df_application.AMT_REQ_CREDIT_BUREAU_DAY[df_application.AMT_REQ_CREDIT_BUREAU_DAY.isna()] = 0
df_application.AMT_REQ_CREDIT_BUREAU_WEEK[df_application.AMT_REQ_CREDIT_BUREAU_WEEK.isna()] = 0
df_application.AMT_REQ_CREDIT_BUREAU_MON[df_application.AMT_REQ_CREDIT_BUREAU_MON.isna()] = 0
df_application.AMT_REQ_CREDIT_BUREAU_QRT[df_application.AMT_REQ_CREDIT_BUREAU_QRT.isna()] = 0
df_application.AMT_REQ_CREDIT_BUREAU_YEAR[df_application.AMT_REQ_CREDIT_BUREAU_YEAR.isna()] = 0
df_application.CNT_FAM_MEMBERS[df_application.CNT_FAM_MEMBERS.isna()] = 0
df_application.OBS_30_CNT_SOCIAL_CIRCLE[df_application.OBS_30_CNT_SOCIAL_CIRCLE.isna()] = 0
df_application.OBS_60_CNT_SOCIAL_CIRCLE[df_application.OBS_60_CNT_SOCIAL_CIRCLE.isna()] = 0
df_application.DEF_30_CNT_SOCIAL_CIRCLE[df_application.DEF_30_CNT_SOCIAL_CIRCLE.isna()] = 0
df_application.DEF_60_CNT_SOCIAL_CIRCLE[df_application.DEF_60_CNT_SOCIAL_CIRCLE.isna()] = 0
df_application.DAYS_LAST_PHONE_CHANGE[df_application.DAYS_LAST_PHONE_CHANGE.isna()] = 0
df_application.DAYS_EMPLOYED[df_application.DAYS_EMPLOYED >0] = 0

<a id="T44"></a>
<h2>   
      <font size="5" , color = #41B6DF>
            5.4. Outliers management
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

In [ ]:
list = [col for col in df_application.columns if col.startswith('AMT')]
list

In [ ]:
GraphOutliers('AMT_INCOME_TOTAL','AMT_INCOME_TOTAL',800000)

In [ ]:
GraphOutliers('AMT_CREDIT','AMT_CREDIT',2000000)

In [ ]:
GraphOutliers('AMT_ANNUITY','AMT_ANNUITY',100000)

In [ ]:
GraphOutliers('AMT_GOODS_PRICE','AMT_GOODS_PRICE',2000000)

Deleting outliers :

In [ ]:
df_application=df_application[df_application.AMT_INCOME_TOTAL<=800000]
df_application=df_application[df_application.AMT_CREDIT<=2000000]
df_application=df_application[df_application.AMT_ANNUITY<=100000]
df_application=df_application[df_application.AMT_GOODS_PRICE<=2000000]

In [ ]:
df_application = df_application [ df_application.CODE_GENDER != "XNA" ]
df_application = df_application [ df_application.FLAG_MOBIL == 1 ]

<a id="T45"></a>
<h2>   
      <font size="5" , color = #41B6DF>
            5.5. New field
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

In [ ]:
df_application['FLAG_DOCUMENT']=df_application['FLAG_DOCUMENT_2']+df_application['FLAG_DOCUMENT_3']+df_application['FLAG_DOCUMENT_4']+df_application['FLAG_DOCUMENT_5']+df_application['FLAG_DOCUMENT_6']+df_application['FLAG_DOCUMENT_7']+df_application['FLAG_DOCUMENT_8']+df_application['FLAG_DOCUMENT_9']+df_application['FLAG_DOCUMENT_10']+df_application['FLAG_DOCUMENT_11']+df_application['FLAG_DOCUMENT_12']+df_application['FLAG_DOCUMENT_13']+df_application['FLAG_DOCUMENT_14']+df_application['FLAG_DOCUMENT_15']+df_application['FLAG_DOCUMENT_16']+df_application['FLAG_DOCUMENT_17']+df_application['FLAG_DOCUMENT_18']+df_application['FLAG_DOCUMENT_19']+df_application['FLAG_DOCUMENT_20']+df_application['FLAG_DOCUMENT_21']

<a id="T5"></a>
<h2>   
      <font size="6" , color = #26B260>
            6. Target
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

In [ ]:
df_application.TARGET.value_counts()

In [ ]:
df_application.TARGET.value_counts(normalize=True) 

<a id="T6"></a>
<h2>   
      <font size="6" , color = #26B260>
            7. Fields gruping
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

Transformation of categorical fields into numeric and new fiels creation :

In [ ]:
for col in df_application.select_dtypes(include=object).columns.tolist() :
    ratios = df_application.groupby(col).apply(lambda x: x['TARGET'].sum() / x['TARGET'].count()).reset_index()
    ratios.columns = [col, 'TEMP']
    ratios=ratios.sort_values(by = 'TEMP')
    liste=ratios[col].values.tolist()
    df_application['X_'+ col] = df_application[col].astype('category')
    df_application['X_'+ col] = df_application['X_'+ col].cat.reorder_categories(liste,ordered=True)
    df_application['X_'+ col] = df_application['X_'+ col].cat.codes
    print('X_'+ col)

Grouping of modalities using decision trees :

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

for col in df_application.select_dtypes(include=np.number).columns.tolist() :
    clf = DecisionTreeClassifier(max_depth=2)
    clf.fit(pd.DataFrame(df_application[col]), pd.DataFrame(df_application['TARGET']))
    #plot_tree(clf)
    #plt.show()
    df_application['CAT_' + col]=clf.apply(pd.DataFrame(df_application[col]))
    df_application['CAT_' + col] = df_application['CAT_' + col].astype(str)
    print('CAT_' + col)

Check on one field : CAT_X_NAME_INCOME_TYPE

In [ ]:
pd.crosstab([df_application["CAT_X_NAME_INCOME_TYPE"],df_application["X_NAME_INCOME_TYPE"],df_application["NAME_INCOME_TYPE"]],
            df_application["TARGET"], margins=True)

<a id="T7"></a>
<h2>   
      <font size="6" , color = #26B260>
            8. Fields analysis
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

<a id="T71"></a>
<h2>   
      <font size="5" , color = #41B6DF>
            8.1. Family
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

Fields studied : 
* CNT_CHILDREN
* NAME_TYPE_SUITE
* NAME_FAMILY_STATUS
* CNT_FAM_MEMBERS
* X_NAME_TYPE_SUITE
* X_NAME_FAMILY_STATUS
* CAT_CNT_CHILDREN
* CAT_CNT_FAM_MEMBERS
* CAT_X_NAME_TYPE_SUITE
* CAT_X_NAME_FAMILY_STATUS

Crossing of family status & nb of children :

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"]}
df_application.groupby(["NAME_FAMILY_STATUS","CAT_X_NAME_FAMILY_STATUS",'CNT_CHILDREN','CAT_CNT_CHILDREN']).agg(aggregations).reset_index()

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"],'CNT_CHILDREN': [min,max]}
df_application.groupby(["CAT_X_NAME_FAMILY_STATUS",'CAT_CNT_CHILDREN']).agg(aggregations).reset_index()

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

clf = DecisionTreeClassifier(max_depth=4)
clf.fit(pd.DataFrame(df_application[["CAT_X_NAME_FAMILY_STATUS","CNT_CHILDREN"]]), pd.DataFrame(df_application['TARGET']))
#plot_tree(clf)
#plt.show()
df_application['CAT_NAME_FAMILY_STATUS_CHILD']=clf.apply(pd.DataFrame(df_application[["CAT_X_NAME_FAMILY_STATUS","CNT_CHILDREN"]]))

# Recoging due to low numbers
df_application["CAT_NAME_FAMILY_STATUS_CHILD"].loc[df_application["CAT_NAME_FAMILY_STATUS_CHILD"]==10]=8
df_application["CAT_NAME_FAMILY_STATUS_CHILD"].loc[df_application["CAT_NAME_FAMILY_STATUS_CHILD"]==11]=8

<div class="alert-info">
        <b>Comments: </b><br>Creation of a new category crossing family status and nb of children is providing more information for the target regarding correlations : CAT_NAME_FAMILY_STATUS_CHILD.   
</div>

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"]}
df_application.groupby(["CAT_X_NAME_TYPE_SUITE","NAME_TYPE_SUITE"]).agg(aggregations).reset_index()

<div class="alert-info">
        <b>Comments: </b><br>The gruping of NAME_TYPE_SUITE is providing 4 groups of members : CAT_X_NAME_TYPE_SUITE.    
</div>

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"]}
df_application.groupby(["CAT_CNT_FAM_MEMBERS","NAME_FAMILY_STATUS"]).agg(aggregations).reset_index()

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"],'CNT_FAM_MEMBERS': [min,"mean",max] }
df_application.groupby(["NAME_FAMILY_STATUS","CNT_CHILDREN"]).agg(aggregations).reset_index()

<div class="alert-info">
<b>Comments: </b><br>Checking the consistancy of CNT_FAM_MEMBERS by crossing family status and nb of children. This field is consistant.    
</div>

We try to create a new field crossing family status, family members and nb of children : 

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

clf = DecisionTreeClassifier(max_depth=6)
clf.fit(pd.DataFrame(df_application[["CAT_X_NAME_FAMILY_STATUS","CNT_FAM_MEMBERS","CNT_CHILDREN"]]), pd.DataFrame(df_application['TARGET']))
plot_tree(clf)
plt.show()
df_application['CAT_NAME_FAMILY_STATUS_MEMBERS']=clf.apply(pd.DataFrame(df_application[["CAT_X_NAME_FAMILY_STATUS","CNT_FAM_MEMBERS","CNT_CHILDREN"]]))

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"],'CNT_FAM_MEMBERS': [min,"mean",max],'CNT_CHILDREN': [min,"mean",max]  }
df_application.groupby(["CAT_NAME_FAMILY_STATUS_MEMBERS","NAME_FAMILY_STATUS"]).agg(aggregations).reset_index()

<div class="alert-info">
<b>Comments: </b><br>Creation of a new filed called CAT_NAME_FAMILY_STATUS_MEMBERS crossing NAME_FAMILY_STATUS, CNT_FAM_MEMBERS and CNT_CHILDREN.    
</div>

Correlations analysis : 

In [ ]:
contact_col = ['CNT_CHILDREN','CNT_FAM_MEMBERS','X_NAME_TYPE_SUITE',
'X_NAME_FAMILY_STATUS','CAT_CNT_CHILDREN','CAT_CNT_FAM_MEMBERS','CAT_X_NAME_TYPE_SUITE','CAT_X_NAME_FAMILY_STATUS',
'CAT_NAME_FAMILY_STATUS_CHILD','CAT_NAME_FAMILY_STATUS_MEMBERS','TARGET']
df_contact = df_application.loc[:,contact_col] 
df_contact = df_contact.astype(float)
df_contact_corr = df_contact.corr()
df_contact_corr

In [ ]:
df_contact_corr = df_contact_corr.where(np.triu(np.ones(df_contact_corr.shape),k=1).astype(np.bool))
df_contact_corr = df_contact_corr.unstack().reset_index()
df_contact_corr.columns =['VAR1','VAR2','Correlation']
df_contact_corr.dropna(subset = ["Correlation"], inplace = True)
df_contact_corr["Correlation"]=df_contact_corr["Correlation"].abs() 
df_contact_corr.sort_values(by='Correlation', ascending=False, inplace=True) 
df_contact_corr

<div class="alert alert-block alert-info">
    <span style='font-family:Georgia'>
        <b>Conclusion: </b><br>High correlation between TARGET and CAT_NAME_FAMILY_STATUS_MEMBERS and CAT_NAME_FAMILY_STATUS_CHILD. We choose to keep CAT_NAME_FAMILY_STATUS_CHILD instead of CAT_NAME_FAMILY_STATUS_MEMBERS for the model construction because it is less difficult to understand due to the nb of modalities and the difference of correlation between CAT_NAME_FAMILY_STATUS_MEMBERS and CAT_NAME_FAMILY_STATUS_CHILD with target is low. NAME_TYPE_SUITE is not providing a significant correlation with TARGET. We keep CAT_NAME_FAMILY_STATUS_CHILD for the model. 
    </span>    
</div>

Fields retained : 
* CAT_NAME_FAMILY_STATUS_CHILD

<a id="T72"></a>
<h2>   
      <font size="5" , color = #41B6DF>
            8.2. Location
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

Fields studied : 
* FLAG_OWN_REALTY
* NAME_HOUSING_TYPE
* REGION_POPULATION_RELATIVE
* REGION_RATING_CLIENT
* REGION_RATING_CLIENT_W_CITY
* X_FLAG_OWN_REALTY
* X_NAME_HOUSING_TYPE
* CAT_REGION_POPULATION_RELATIVE
* CAT_REGION_RATING_CLIENT
* CAT_REGION_RATING_CLIENT_W_CITY
* CAT_X_FLAG_OWN_REALTY
* CAT_X_NAME_HOUSING_TYPE

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"]}
df_application.groupby(["CAT_REGION_POPULATION_RELATIVE"]).agg(aggregations).reset_index()

<div class="alert-info">
<b>Comments: </b><br>Logical gruping of fields for CAT_REGION_POPULATION_RELATIVE. Testing with more grups is not providing more significant information.   
</div>

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"]}
df_application.groupby(["CAT_REGION_RATING_CLIENT"]).agg(aggregations).reset_index()

<div class="alert-info">
<b>Comments: </b><br>Logical gruping of fields for CAT_REGION_RATING_CLIENT. Testing with more grups is not providing more significant information.   
</div>

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"]}
df_application.groupby(["FLAG_OWN_REALTY"]).agg(aggregations).reset_index()

In [ ]:
TestHomog('FLAG_OWN_REALTY')

<div class="alert-info">
<b>Comments: </b><br>Difference of target is not showing important difference between having or not a house but the test is showing the contrary.    
</div>

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"]}
df_application.groupby(["CAT_X_NAME_HOUSING_TYPE","NAME_HOUSING_TYPE"]).agg(aggregations).reset_index()

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"]}
df_application.groupby(["CAT_X_NAME_HOUSING_TYPE"]).agg(aggregations).reset_index()

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

clf = DecisionTreeClassifier(max_depth=4)
clf.fit(pd.DataFrame(df_application[["CAT_X_NAME_HOUSING_TYPE","CAT_X_FLAG_OWN_REALTY"]]), pd.DataFrame(df_application['TARGET']))
plot_tree(clf)
plt.show()
df_application['CAT_NAME_HOUSING_TYPE_OWN_REALTY']=clf.apply(pd.DataFrame(df_application[["CAT_X_NAME_HOUSING_TYPE","CAT_X_FLAG_OWN_REALTY"]]))

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"]}
df_application.groupby(["CAT_NAME_HOUSING_TYPE_OWN_REALTY","FLAG_OWN_REALTY","NAME_HOUSING_TYPE"]).agg(aggregations).reset_index()

<div class="alert-info">
<b>Comments: </b><br>As FLAG_OWN_REALTY and NAME_HOUSING_TYPE are showing different TARGET values by categories, we try to build a new field mixing both CAT_NAME_HOUSING_TYPE_OWN_REALTY.    
</div>

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"]}
df_application.groupby(["CAT_REGION_RATING_CLIENT_W_CITY"]).agg(aggregations).reset_index()

Correlation analysis : 

In [ ]:
contact_col = ['REGION_POPULATION_RELATIVE','REGION_RATING_CLIENT',
'REGION_RATING_CLIENT_W_CITY','X_FLAG_OWN_REALTY','X_NAME_HOUSING_TYPE','CAT_REGION_POPULATION_RELATIVE',
'CAT_REGION_RATING_CLIENT','CAT_REGION_RATING_CLIENT_W_CITY','CAT_X_FLAG_OWN_REALTY','CAT_X_NAME_HOUSING_TYPE',
'CAT_NAME_HOUSING_TYPE_OWN_REALTY','TARGET']
df_contact = df_application.loc[:,contact_col] 
df_contact = df_contact.astype(float)
df_contact_corr = df_contact.corr()
df_contact_corr

In [ ]:
df_contact_corr = df_contact_corr.where(np.triu(np.ones(df_contact_corr.shape),k=1).astype(np.bool))
df_contact_corr = df_contact_corr.unstack().reset_index()
df_contact_corr.columns =['VAR1','VAR2','Correlation']
df_contact_corr.dropna(subset = ["Correlation"], inplace = True)
df_contact_corr["Correlation"]=df_contact_corr["Correlation"].abs() 
df_contact_corr.sort_values(by='Correlation', ascending=False, inplace=True) 
df_contact_corr.head(50)

<div class="alert alert-block alert-info">
    <span style='font-family:Georgia'>
        <b>Conclusion: </b><br>TARGET is highly correlated to CAT_REGION_RATING_CLIENT_W_CITY and after with CAT_REGION_RATING_CLIENT / CAT_REGION_POPULATION_RELATIVE (but the last ones are correlated together). So keeping information from CAT_REGION_RATING_CLIENT_W_CITY is sufficient. CAT_X_NAME_HOUSING_TYPE is also highly correlated to TARGET and has low correlation with CAT_REGION_RATING_CLIENT_W_CITY.
    </span>    
</div>

Fields retained : 
* CAT_REGION_RATING_CLIENT_W_CITY
* CAT_X_NAME_HOUSING_TYPE

<a id="T73"></a>
<h2>   
      <font size="5" , color = #41B6DF>
            8.3. Revenues / Job
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

Fields studied : 
* AMT_INCOME_TOTAL
* NAME_INCOME_TYPE
* NAME_EDUCATION_TYPE
* OCCUPATION_TYPE
* ORGANIZATION_TYPE
* X_NAME_INCOME_TYPE
* X_NAME_EDUCATION_TYPE
* X_OCCUPATION_TYPE
* X_ORGANIZATION_TYPE
* CAT_AMT_INCOME_TOTAL
* CAT_X_NAME_INCOME_TYPE
* CAT_X_NAME_EDUCATION_TYPE
* CAT_X_OCCUPATION_TYPE
* CAT_X_ORGANIZATION_TYPE
* CAT_DAYS_EMPLOYED
* DAYS_EMPLOYED

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"], 'AMT_INCOME_TOTAL': [min,"mean",max]}
df_application.groupby(["CAT_AMT_INCOME_TOTAL"]).agg(aggregations).reset_index()

<div class="alert-info">
<b>Comments: </b><br>Logical gruping of CAT_AMT_INCOME_TOTAL. No meaning for higher number of classes.  
</div>

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"], 'AMT_INCOME_TOTAL': ["mean"]}
df_application.groupby(["CAT_X_NAME_INCOME_TYPE","NAME_INCOME_TYPE"]).agg(aggregations).reset_index()

<div class="alert-info">
<b>Comments: </b><br>Logical gruping of CAT_X_NAME_INCOME_TYPE. No meaning for higher number of classes.  
</div>

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

clf = DecisionTreeClassifier(max_depth=4)
clf.fit(pd.DataFrame(df_application[["CAT_X_NAME_INCOME_TYPE","AMT_INCOME_TOTAL"]]), pd.DataFrame(df_application['TARGET']))
plot_tree(clf)
plt.show()
df_application['CAT_NAME_INCOME_TYPE_INCOME']=clf.apply(pd.DataFrame(df_application[["CAT_X_NAME_INCOME_TYPE","AMT_INCOME_TOTAL"]]))

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"],'AMT_INCOME_TOTAL': [min,"mean",max]}
df_application.groupby(["CAT_NAME_INCOME_TYPE_INCOME"]).agg(aggregations).reset_index()

<div class="alert-info">
<b>Comments: </b><br>Test of crossing NAME_INCOME_TYPE & AMT_INCOME by creating the filed CAT_NAME_INCOME_TYPE_INCOME.  
</div>

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"], 'AMT_INCOME_TOTAL': ["mean"]}
df_application.groupby(["CAT_X_NAME_INCOME_TYPE","CAT_DAYS_EMPLOYED"]).agg(aggregations).reset_index()

<div class="alert-info">
<b>Comments: </b><br>Test of crossing NAME_INCOME_TYPE & DAYS_EMPLOYED.  
</div>

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"], 'AMT_INCOME_TOTAL': ["mean"]}
df_application.groupby(["CAT_X_ORGANIZATION_TYPE"]).agg(aggregations).reset_index()

<div class="alert-info">
<b>Comments: </b><br>A lot of organization types are showing logical gruping.
</div>

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"], 'DAYS_EMPLOYED': ["mean"]}
df_application.groupby(["CAT_DAYS_EMPLOYED"]).agg(aggregations).reset_index()

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

clf = DecisionTreeClassifier(max_depth=4)
clf.fit(pd.DataFrame(df_application[["CAT_X_NAME_INCOME_TYPE","DAYS_EMPLOYED"]]), pd.DataFrame(df_application['TARGET']))
plot_tree(clf)
plt.show()
df_application['CAT_NAME_INCOME_TYPE_DAYS_EMPLOYED']=clf.apply(pd.DataFrame(df_application[["CAT_X_NAME_INCOME_TYPE","DAYS_EMPLOYED"]]))

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"], 'DAYS_EMPLOYED': ["mean"]}
df_application.groupby(["CAT_NAME_INCOME_TYPE_DAYS_EMPLOYED"]).agg(aggregations).reset_index()

<div class="alert-info">
<b>Comments: </b><br>Creation of a new field CAT_NAME_INCOME_TYPE_DAYS_EMPLOYED which is a mix between DAYS_EMPLOYED and NAME INCOME TYPE.
</div>

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"]}
df_application.groupby(["CAT_X_ORGANIZATION_TYPE"]).agg(aggregations).reset_index()

Correlations analysis : 

In [ ]:
contact_col = ['AMT_INCOME_TOTAL','X_NAME_INCOME_TYPE','X_NAME_EDUCATION_TYPE','X_OCCUPATION_TYPE','X_ORGANIZATION_TYPE',
'CAT_AMT_INCOME_TOTAL','CAT_X_NAME_INCOME_TYPE','CAT_X_NAME_EDUCATION_TYPE','CAT_X_OCCUPATION_TYPE',
'CAT_X_ORGANIZATION_TYPE','CAT_NAME_INCOME_TYPE_INCOME','CAT_DAYS_EMPLOYED','DAYS_EMPLOYED','CAT_NAME_INCOME_TYPE_DAYS_EMPLOYED','TARGET']
df_contact = df_application.loc[:,contact_col] 
df_contact = df_contact.astype(float)
df_contact_corr = df_contact.corr()
df_contact_corr

In [ ]:
df_contact_corr = df_contact_corr.where(np.triu(np.ones(df_contact_corr.shape),k=1).astype(np.bool))
df_contact_corr = df_contact_corr.unstack().reset_index()
df_contact_corr.columns =['VAR1','VAR2','Correlation']
df_contact_corr.dropna(subset = ["Correlation"], inplace = True)
df_contact_corr["Correlation"]=df_contact_corr["Correlation"].abs() 
df_contact_corr.sort_values(by='Correlation', ascending=False, inplace=True)
df_contact_corr.loc[df_contact_corr.VAR1=="TARGET"]

In [ ]:
contact_col = ['CAT_X_OCCUPATION_TYPE','X_ORGANIZATION_TYPE','CAT_X_NAME_INCOME_TYPE',
'CAT_X_NAME_EDUCATION_TYPE','CAT_AMT_INCOME_TOTAL','CAT_DAYS_EMPLOYED','CAT_NAME_INCOME_TYPE_DAYS_EMPLOYED']
df_contact = df_application.loc[:,contact_col] 
df_contact_corr = df_contact.corr()
df_contact_corr = df_contact_corr.where(np.triu(np.ones(df_contact_corr.shape),k=1).astype(np.bool))
df_contact_corr = df_contact_corr.unstack().reset_index()
df_contact_corr.columns =['VAR1','VAR2','Correlation']
df_contact_corr.dropna(subset = ["Correlation"], inplace = True)
df_contact_corr["Correlation"]=df_contact_corr["Correlation"].abs() 
df_contact_corr.sort_values(by='Correlation', ascending=False, inplace=True)
df_contact_corr

<div class="alert alert-block alert-info">
    <span style='font-family:Georgia'>
        <b>Conclusion: </b><br>High corretation between TARGET and all categories (CAT_NAME_INCOME_TYPE_DAYS_EMPLOYED, CAT_X_OCCUPATION_TYPE,X_ORGANIZATION_TYPE,CAT_X_NAME_INCOME_TYPE,CAT_X_NAME_EDUCATION_TYPE,CAT_AMT_INCOME_TOTAL). According to the fact some of them are correlated together, the fields kept are : CAT_NAME_INCOME_TYPE_DAYS_EMPLOYED and CAT_X_NAME_EDUCATION_TYPE. 
    </span>    
</div>

Fields retained : 
* CAT_NAME_INCOME_TYPE_DAYS_EMPLOYED
* CAT_X_NAME_EDUCATION_TYPE

<a id="T74"></a>
<h2>   
      <font size="5" , color = #41B6DF>
            8.4. Contact variables
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

Fields studied : 
* FLAG_MOBIL
* FLAG_EMP_PHONE
* FLAG_WORK_PHONE
* FLAG_CONT_MOBILE
* FLAG_PHONE
* FLAG_EMAIL
* DAYS_LAST_PHONE_CHANGE
* CAT_FLAG_MOBIL
* CAT_FLAG_EMP_PHONE
* CAT_FLAG_WORK_PHONE
* CAT_FLAG_CONT_MOBILE
* CAT_FLAG_PHONE
* CAT_FLAG_EMAIL
* CAT_DAYS_LAST_PHONE_CHANGE

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"],'DAYS_LAST_PHONE_CHANGE': [min,max],}
df_application.groupby(["CAT_" + 'DAYS_LAST_PHONE_CHANGE']).agg(aggregations).reset_index()

<div class="alert-info">
<b>Comments: </b><br>More the phone number has been modified recently, more the customers are in default.
</div>

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"]}
df_application.groupby(["CAT_" + 'FLAG_MOBIL','FLAG_MOBIL']).agg(aggregations).reset_index()

<div class="alert-info">
<b>Comments: </b><br>Every customer has provided a mobil number. This field will not be studied in following analysis. 
</div>

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"]}
df_application.groupby(['FLAG_WORK_PHONE']).agg(aggregations).reset_index()

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

clf = DecisionTreeClassifier(max_depth=4)
clf.fit(pd.DataFrame(df_application[["X_NAME_INCOME_TYPE",'CAT_FLAG_WORK_PHONE']]), pd.DataFrame(df_application['TARGET']))
plot_tree(clf)
plt.show()
df_application['CAT_NAME_INCOME_TYPE_WORK_PHONE']=clf.apply(pd.DataFrame(df_application[["X_NAME_INCOME_TYPE",'CAT_FLAG_WORK_PHONE']]))

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"]}
df_application.groupby(["CAT_NAME_INCOME_TYPE_WORK_PHONE","NAME_INCOME_TYPE",'FLAG_WORK_PHONE']).agg(aggregations).reset_index()

<div class="alert-info">
<b>Comments: </b><br>Creation of new field CAT_NAME_INCOME_TYPE_WORK_PHONE crossing NAME_INCOME_TYPE and WORK PHONE to exclude customers without job impact on the WORK PHONE field.
</div>

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"]}
df_application.groupby(['FLAG_EMP_PHONE']).agg(aggregations).reset_index()

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

clf = DecisionTreeClassifier(max_depth=4)
clf.fit(pd.DataFrame(df_application[["X_NAME_INCOME_TYPE",'CAT_FLAG_EMP_PHONE']]), pd.DataFrame(df_application['TARGET']))
plot_tree(clf)
plt.show()
df_application['CAT_NAME_INCOME_TYPE_EMP_PHONE']=clf.apply(pd.DataFrame(df_application[["X_NAME_INCOME_TYPE",'CAT_FLAG_EMP_PHONE']]))

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"]}
df_application.groupby(["CAT_NAME_INCOME_TYPE_EMP_PHONE","NAME_INCOME_TYPE",'FLAG_EMP_PHONE']).agg(aggregations).reset_index()

<div class="alert-info">
<b>Comments: </b><br>Creation of new field CAT_NAME_INCOME_TYPE_EMP_PHONE crossing NAME_INCOME_TYPE and EMP PHONE to exclude customers without job impact on the EMP PHONE field. . 
</div>

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"]}
df_application.groupby(['FLAG_CONT_MOBILE']).agg(aggregations).reset_index()

In [ ]:
TestHomog('FLAG_CONT_MOBILE')

<div class="alert-info">
<b>Comments: </b><br>Small number of customers without a cont mobile phone number. Test is not showing there is a difference between two proportions. 
</div>

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"]}
df_application.groupby(['FLAG_PHONE']).agg(aggregations).reset_index()

In [ ]:
TestHomog('FLAG_PHONE')

<div class="alert-info">
<b>Comments: </b><br>Difference is shown between customer with or without phone number. 
</div>

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"]}
df_application.groupby(["FLAG_EMAIL"]).agg(aggregations).reset_index()

In [ ]:
TestHomog('FLAG_EMAIL')

<div class="alert-info">
<b>Comments: </b><br>Small difference of default for customers with or without email address. 
</div>

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

clf = DecisionTreeClassifier(max_depth=4)
clf.fit(pd.DataFrame(df_application[["CAT_FLAG_PHONE",'CAT_FLAG_EMAIL']]), pd.DataFrame(df_application['TARGET']))
plot_tree(clf)
plt.show()
df_application['CAT_PHONE_EMAIL']=clf.apply(pd.DataFrame(df_application[["CAT_FLAG_PHONE",'CAT_FLAG_EMAIL']]))

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"]}
df_application.groupby(["CAT_PHONE_EMAIL"]).agg(aggregations).reset_index()

<div class="alert-info">
<b>Comments: </b><br>Creation of a new field CAT_PHONE_EMAIL mix between phone and email. 
</div>

Correlation analysis : 

In [ ]:
contact_col = ['FLAG_EMP_PHONE','FLAG_WORK_PHONE','FLAG_CONT_MOBILE','FLAG_PHONE','FLAG_EMAIL',
               'DAYS_LAST_PHONE_CHANGE','CAT_FLAG_MOBIL','CAT_FLAG_EMP_PHONE','CAT_FLAG_WORK_PHONE',
               'CAT_FLAG_CONT_MOBILE','CAT_FLAG_PHONE','CAT_FLAG_EMAIL','CAT_DAYS_LAST_PHONE_CHANGE',
               'CAT_NAME_INCOME_TYPE_WORK_PHONE','CAT_NAME_INCOME_TYPE_EMP_PHONE','CAT_PHONE_EMAIL','TARGET']
df_contact = df_application.loc[:,contact_col] 
df_contact = df_contact.astype(float)
df_contact_corr = df_contact.corr()
df_contact_corr

In [ ]:
df_contact_corr = df_contact_corr.where(np.triu(np.ones(df_contact_corr.shape),k=1).astype(np.bool))
df_contact_corr = df_contact_corr.unstack().reset_index()
df_contact_corr.columns =['VAR1','VAR2','Correlation']
df_contact_corr.dropna(subset = ["Correlation"], inplace = True)
df_contact_corr["Correlation"]=df_contact_corr["Correlation"].abs() 
df_contact_corr.sort_values(by='Correlation', ascending=False, inplace=True) 
df_contact_corr.loc[df_contact_corr.VAR1=="TARGET"]

In [ ]:
df_contact_corr

<div class="alert alert-block alert-info">
    <span style='font-family:Georgia'>
        <b>Conclusion: </b><br>Higher corretation between TARGET and CAT_NAME_INCOME_TYPE_WORK_PHONE and CAT_DAYS_LAST_PHONE_CHANGE. CAT_NAME_INCOME_TYPE_EMP_PHONE has also a correlation with target but is linked to CAT_NAME_INCOME_TYPE_WORK_PHONE. CAT_PHONE_EMAIL is kept in the following analysis (corr with target 0.023 but could have a meaning). 
    </span>    
</div>

Fields retained : 
* CAT_NAME_INCOME_TYPE_WORK_PHONE
* CAT_DAYS_LAST_PHONE_CHANGE
* CAT_PHONE_EMAIL

<a id="T75"></a>
<h2>   
      <font size="5" , color = #41B6DF>
            8.5. Loan variables
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

Fields studied : 
* WEEKDAY_APPR_PROCESS_START
* HOUR_APPR_PROCESS_START
* REG_REGION_NOT_LIVE_REGION
* REG_REGION_NOT_WORK_REGION
* LIVE_REGION_NOT_WORK_REGION
* REG_CITY_NOT_LIVE_CITY
* REG_CITY_NOT_WORK_CITY
* LIVE_CITY_NOT_WORK_CITY
* EXT_SOURCE_2
* EXT_SOURCE_3
* OBS_30_CNT_SOCIAL_CIRCLE
* DEF_30_CNT_SOCIAL_CIRCLE
* OBS_60_CNT_SOCIAL_CIRCLE
* DEF_60_CNT_SOCIAL_CIRCLE
* FLAG_DOCUMENT_X
* AMT_REQ_CREDIT_BUREAU_HOUR
* AMT_REQ_CREDIT_BUREAU_DAY
* AMT_REQ_CREDIT_BUREAU_WEEK
* AMT_REQ_CREDIT_BUREAU_MON
* AMT_REQ_CREDIT_BUREAU_QRT
* AMT_REQ_CREDIT_BUREAU_YEAR
* FLAG_DOCUMENT
* X_WEEKDAY_APPR_PROCESS_START
* CAT_HOUR_APPR_PROCESS_START
* CAT_REG_REGION_NOT_LIVE_REGION
* CAT_REG_REGION_NOT_WORK_REGION
* CAT_LIVE_REGION_NOT_WORK_REGION
* CAT_REG_CITY_NOT_LIVE_CITY
* CAT_REG_CITY_NOT_WORK_CITY
* CAT_LIVE_CITY_NOT_WORK_CITY
* CAT_EXT_SOURCE_2
* CAT_EXT_SOURCE_3
* CAT_OBS_30_CNT_SOCIAL_CIRCLE
* CAT_DEF_30_CNT_SOCIAL_CIRCLE
* CAT_OBS_60_CNT_SOCIAL_CIRCLE
* CAT_DEF_60_CNT_SOCIAL_CIRCLE
* CAT_FLAG_DOCUMENT_X
* CAT_AMT_REQ_CREDIT_BUREAU_HOUR
* CAT_AMT_REQ_CREDIT_BUREAU_DAY
* CAT_AMT_REQ_CREDIT_BUREAU_WEEK
* CAT_AMT_REQ_CREDIT_BUREAU_MON
* CAT_AMT_REQ_CREDIT_BUREAU_QRT
* CAT_AMT_REQ_CREDIT_BUREAU_YEAR
* CAT_FLAG_DOCUMENT
* CAT_X_WEEKDAY_APPR_PROCESS_START

Regarding the number of field, the analysis will start with correlation analysis. 

In [ ]:
contact_col = ['CAT_HOUR_APPR_PROCESS_START','CAT_REG_REGION_NOT_LIVE_REGION','CAT_REG_REGION_NOT_WORK_REGION',
               'CAT_LIVE_REGION_NOT_WORK_REGION','CAT_REG_CITY_NOT_LIVE_CITY','CAT_REG_CITY_NOT_WORK_CITY','CAT_LIVE_CITY_NOT_WORK_CITY',
               'CAT_EXT_SOURCE_2','CAT_EXT_SOURCE_3','CAT_FLAG_DOCUMENT_2','CAT_FLAG_DOCUMENT_3','CAT_FLAG_DOCUMENT_4','CAT_FLAG_DOCUMENT_5',
               'CAT_FLAG_DOCUMENT_6','CAT_FLAG_DOCUMENT_7','CAT_FLAG_DOCUMENT_8','CAT_FLAG_DOCUMENT_9','CAT_FLAG_DOCUMENT_10',
               'CAT_FLAG_DOCUMENT_11','CAT_FLAG_DOCUMENT_12','CAT_FLAG_DOCUMENT_13','CAT_FLAG_DOCUMENT_14','CAT_FLAG_DOCUMENT_15',
               'CAT_FLAG_DOCUMENT_16','CAT_FLAG_DOCUMENT_17','CAT_FLAG_DOCUMENT_18','CAT_FLAG_DOCUMENT_19','CAT_FLAG_DOCUMENT_20',
               'CAT_FLAG_DOCUMENT_21','CAT_AMT_REQ_CREDIT_BUREAU_HOUR','CAT_AMT_REQ_CREDIT_BUREAU_DAY','CAT_AMT_REQ_CREDIT_BUREAU_WEEK',
               'CAT_AMT_REQ_CREDIT_BUREAU_MON','CAT_AMT_REQ_CREDIT_BUREAU_QRT','CAT_AMT_REQ_CREDIT_BUREAU_YEAR','CAT_FLAG_DOCUMENT',
               'CAT_X_WEEKDAY_APPR_PROCESS_START','TARGET']
df_contact = df_application.loc[:,contact_col] 
df_contact = df_contact.astype(float)
df_contact_corr = df_contact.corr()
df_contact_corr

In [ ]:
df_contact_corr = df_contact_corr.where(np.triu(np.ones(df_contact_corr.shape),k=1).astype(np.bool))
df_contact_corr = df_contact_corr.unstack().reset_index()
df_contact_corr.columns =['VAR1','VAR2','Correlation']
df_contact_corr.dropna(subset = ["Correlation"], inplace = True)
df_contact_corr["Correlation"]=df_contact_corr["Correlation"].abs() 
df_contact_corr.sort_values(by='Correlation', ascending=False, inplace=True) 
df_contact_corr.loc[df_contact_corr.VAR1=="TARGET"]

In [ ]:
contact_col = ['CAT_HOUR_APPR_PROCESS_START','CAT_REG_CITY_NOT_LIVE_CITY','CAT_REG_CITY_NOT_WORK_CITY',
               'CAT_LIVE_CITY_NOT_WORK_CITY','CAT_EXT_SOURCE_2','CAT_EXT_SOURCE_3','CAT_FLAG_DOCUMENT_3',
               'CAT_FLAG_DOCUMENT_6','CAT_FLAG_DOCUMENT_13','CAT_FLAG_DOCUMENT_16','CAT_AMT_REQ_CREDIT_BUREAU_MON',
               'CAT_AMT_REQ_CREDIT_BUREAU_QRT','CAT_FLAG_DOCUMENT','TARGET']

df_contact = df_application.loc[:,contact_col] 
df_contact_corr = df_contact.corr()
df_contact_corr

In [ ]:
df_contact_corr = df_contact_corr.where(np.triu(np.ones(df_contact_corr.shape),k=1).astype(np.bool))
df_contact_corr = df_contact_corr.unstack().reset_index()
df_contact_corr.columns =['VAR1','VAR2','Correlation']
df_contact_corr.dropna(subset = ["Correlation"], inplace = True)
df_contact_corr["Correlation"]=df_contact_corr["Correlation"].abs() 
df_contact_corr.sort_values(by='Correlation', ascending=False, inplace=True) 
df_contact_corr.head(50)

<div class="alert alert-block alert-info">
    <span>
        <b>Conclusion: </b><br>Following fields are selected based on analysis between TARGET and between them themself : CAT_REG_CITY_NOT_WORK_CITY, CAT_EXT_SOURCE_2, CAT_EXT_SOURCE_3, CAT_FLAG_DOCUMENT_3
    </span>    
</div>

Fields retained : 
* CAT_REG_CITY_NOT_WORK_CITY
* CAT_EXT_SOURCE_2
* CAT_EXT_SOURCE_3
* CAT_FLAG_DOCUMENT_3

<a id="T76"></a>
<h2>   
      <font size="5" , color = #41B6DF>
            8.6. Other
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

Fields studied : 
* NAME_CONTRACT_TYPE
* FLAG_OWN_CAR
* AMT_CREDIT
* AMT_ANNUITY
* AMT_GOODS_PRICE
* DAYS_BIRTH
* DAYS_REGISTRATION
* DAYS_ID_PUBLISH
* X_NAME_CONTRACT_TYPE
* X_FLAG_OWN_CAR
* CAT_AMT_CREDIT
* CAT_AMT_ANNUITY
* CAT_AMT_GOODS_PRICE
* CAT_DAYS_BIRTH
* CAT_DAYS_REGISTRATION
* CAT_DAYS_ID_PUBLISH
* CAT_X_NAME_CONTRACT_TYPE
* CAT_X_FLAG_OWN_CAR

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"]}
df_application.groupby(['CAT_X_NAME_CONTRACT_TYPE']).agg(aggregations).reset_index()

In [ ]:
TestHomog('CAT_X_NAME_CONTRACT_TYPE')

<div class="alert-info">
<b>Comments: </b><br>It is shown an impact of the contract type on target. 
</div>

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"],'DAYS_BIRTH': [min,"mean",max]}
df_application.groupby(['CAT_DAYS_BIRTH']).agg(aggregations).reset_index()

<div class="alert-info">
<b>Comments: </b><br>TARGET and age are linked. 
</div>

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"]}
df_application.groupby(['CAT_DAYS_ID_PUBLISH']).agg(aggregations).reset_index()

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"]}
df_application.groupby(['CAT_DAYS_REGISTRATION']).agg(aggregations).reset_index()

<div class="alert-info">
<b>Comments: </b><br>DAYS_ID_PUBLISH & DAYS_REGISTRATION seems linked to target. 
</div>

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"],'AMT_GOODS_PRICE': [min,"mean",max]}
df_application.groupby(['CAT_AMT_GOODS_PRICE']).agg(aggregations).reset_index()

<div class="alert-info">
<b>Comments: </b><br>Higher default proportion is observed on the second category [342 K; 450 K] and after on lower amount good prices. Without this strage impact, default rated is decreasing when good price increases. 
</div>

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

clf = DecisionTreeClassifier(max_depth=4)
clf.fit(pd.DataFrame(df_application[["AMT_GOODS_PRICE",'AMT_CREDIT']]), pd.DataFrame(df_application['TARGET']))
plot_tree(clf)
plt.show()
df_application['CAT_GOODS_CREDIT']=clf.apply(pd.DataFrame(df_application[["AMT_GOODS_PRICE",'AMT_CREDIT']]))

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"],'AMT_GOODS_PRICE': [min,"mean",max]}
df_application.groupby(['CAT_GOODS_CREDIT']).agg(aggregations).reset_index()

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

clf = DecisionTreeClassifier(max_depth=4)
clf.fit(pd.DataFrame(df_application[["AMT_CREDIT",'AMT_ANNUITY']]), pd.DataFrame(df_application['TARGET']))
plot_tree(clf)
plt.show()
df_application['CAT_CREDIT_ANNUITY']=clf.apply(pd.DataFrame(df_application[["AMT_CREDIT",'AMT_ANNUITY']]))

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"],'AMT_GOODS_PRICE': [min,"mean",max]}
df_application.groupby(['CAT_CREDIT_ANNUITY']).agg(aggregations).reset_index()

<div class="alert-info">
<b>Comments: </b><br>Creation of new fields with AMT_CREDIT, AMT_GOOD_PRICE, AMT_ANNUITY to study the impact on target. 
</div>

In [ ]:
aggregations = {'TARGET': [sum,'count',"mean"]}
df_application.groupby(['CAT_X_FLAG_OWN_CAR']).agg(aggregations).reset_index()

In [ ]:
TestHomog('FLAG_OWN_CAR')

<div class="alert-info">
<b>Comments: </b><br>FLAG_OWN_CAR is showing to have an impact on the default level. 
</div>

Correlation analysis : 

In [ ]:
contact_col = ['AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE','DAYS_BIRTH',
               'DAYS_REGISTRATION','DAYS_ID_PUBLISH','X_NAME_CONTRACT_TYPE','X_FLAG_OWN_CAR','CAT_AMT_CREDIT',
               'CAT_AMT_ANNUITY','CAT_AMT_GOODS_PRICE','CAT_DAYS_BIRTH','CAT_DAYS_REGISTRATION','CAT_DAYS_ID_PUBLISH',
               'CAT_X_NAME_CONTRACT_TYPE','CAT_X_FLAG_OWN_CAR','CAT_GOODS_CREDIT','CAT_CREDIT_ANNUITY','TARGET']
df_contact = df_application.loc[:,contact_col] 
df_contact = df_contact.astype(float)
df_contact_corr = df_contact.corr()
df_contact_corr

In [ ]:
df_contact_corr = df_contact_corr.where(np.triu(np.ones(df_contact_corr.shape),k=1).astype(np.bool))
df_contact_corr = df_contact_corr.unstack().reset_index()
df_contact_corr.columns =['VAR1','VAR2','Correlation']
df_contact_corr.dropna(subset = ["Correlation"], inplace = True)
df_contact_corr["Correlation"]=df_contact_corr["Correlation"].abs() 
df_contact_corr.sort_values(by='Correlation', ascending=False, inplace=True) 
df_contact_corr.head(50)

In [ ]:
df_contact_corr.loc[df_contact_corr.VAR1=="TARGET"]

In [ ]:
contact_col = ['CAT_AMT_CREDIT','CAT_AMT_ANNUITY','CAT_AMT_GOODS_PRICE','AMT_CREDIT','AMT_ANNUITY',
               'AMT_GOODS_PRICE','TARGET']
df_contact = df_application.loc[:,contact_col] 
df_contact_corr = df_contact.corr()
df_contact_corr

In [ ]:
df_contact_corr = df_contact_corr.where(np.triu(np.ones(df_contact_corr.shape),k=1).astype(np.bool))
df_contact_corr = df_contact_corr.unstack().reset_index()
df_contact_corr.columns =['VAR1','VAR2','Correlation']
df_contact_corr.dropna(subset = ["Correlation"], inplace = True)
df_contact_corr["Correlation"]=df_contact_corr["Correlation"].abs() 
df_contact_corr.sort_values(by='Correlation', ascending=False, inplace=True) 
df_contact_corr.head(50)

<div class="alert alert-block alert-info">
    <span>
        <b>Conclusion: </b><br>Following fields are selected based on analysis between TARGET and between them themself : CAT_AMT_GOODS_PRICE, CAT_DAYS_BIRTH, CAT_DAYS_REGISTRATION, CAT_DAYS_ID_PUBLISH, CAT_X_NAME_CONTRACT_TYPE, CAT_X_FLAG_OWN_CAR. The new fields created with amount fields are not showing importance. 
    </span>    
</div>

Fields retained : 
* CAT_AMT_GOODS_PRICE
* CAT_DAYS_BIRTH
* CAT_DAYS_REGISTRATION
* CAT_DAYS_ID_PUBLISH
* CAT_X_NAME_CONTRACT_TYPE
* CAT_X_FLAG_OWN_CAR

<a id="T8"></a>
<h2>   
      <font size="6" , color = #26B260>
            9. Final fields selection
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

In [ ]:
contact_col = ['CAT_AMT_GOODS_PRICE','CAT_DAYS_BIRTH','CAT_DAYS_REGISTRATION','CAT_DAYS_ID_PUBLISH',
               'CAT_REGION_RATING_CLIENT_W_CITY','CAT_REG_CITY_NOT_WORK_CITY','CAT_EXT_SOURCE_2',
               'CAT_EXT_SOURCE_3','CAT_DAYS_LAST_PHONE_CHANGE','CAT_FLAG_DOCUMENT_3','CAT_X_NAME_CONTRACT_TYPE',
               'CAT_X_FLAG_OWN_CAR','CAT_X_NAME_EDUCATION_TYPE','CAT_X_NAME_HOUSING_TYPE','CAT_NAME_FAMILY_STATUS_CHILD',
               'CAT_NAME_INCOME_TYPE_WORK_PHONE', 'CAT_NAME_INCOME_TYPE_DAYS_EMPLOYED','CAT_PHONE_EMAIL','TARGET']
df_contact = df_application.loc[:,contact_col] 
df_contact_corr = df_contact.corr()
df_contact_corr

In [ ]:
df_contact_corr = df_contact_corr.where(np.triu(np.ones(df_contact_corr.shape),k=1).astype(np.bool))
df_contact_corr = df_contact_corr.unstack().reset_index()
df_contact_corr.columns =['VAR1','VAR2','Correlation']
df_contact_corr.dropna(subset = ["Correlation"], inplace = True)
df_contact_corr["Correlation"]=df_contact_corr["Correlation"].abs() 
df_contact_corr.sort_values(by='Correlation', ascending=False, inplace=True) 
df_contact_corr.head(50)

In [ ]:
df_contact_corr.loc[df_contact_corr.VAR1=="TARGET"]

<div class="alert alert-block alert-info">
    <span>
        <b>Conclusion: </b><br>CAT_X_NAME_CONTRACT_TYPE & CAT_FLAG_DOCUMENT_3 are linked at 0.5 meaning that Document3 is asked for a specific contract type. We delete CAT_FLAG_DOCUMENT_3 from the list of selection. CAT_NAME_INCOME_TYPE_DAYS_EMPLOYED & CAT_NAME_INCOME_TYPE_WORK_PHONE are linked together at 0.88. We delete CAT_NAME_INCOME_TYPE_WORK_PHONE from the list of selection. CAT_DAYS_BIRTH is linked to CAT_DAYS_REGISTRATION, CAT_NAME_FAMILY_STATUS_CHILD	, CAT_DAYS_ID_PUBLISH at 0.3. We delete CAT_DAYS_BIRTH from the list of selection even if it is more linked to TARGET.
    </span>    
</div>

Calculation of WoE and IV on selected features : 

In [ ]:
columns = [c for c in df_application]
columns.remove('SK_ID_CURR')

df_XY =df_application.loc[:,columns]

In [ ]:
ListeNom=[]
ListeIv=[]
ListeIvCom=[]
i=1
for col in df_XY.select_dtypes(include=object).columns.tolist():
    df, iv = calculate_woe_iv(df_XY, col, "TARGET")
    print(col)
    print(df)
    print('IV score: {:.2f}'.format(iv)+" - " + IV_Comment (iv))
    print(str(i)+"/"+str(len(df_XY.select_dtypes(include=object).columns.tolist())))
    print("")
    ListeNom.append(col)
    ListeIv.append(iv)
    ListeIvCom.append(IV_Comment (iv))
    i=i+1

In [ ]:
df_IV = pd.DataFrame(data={'Nom':ListeNom,'IV': ListeIv,'IV comment': ListeIvCom})#
df_IV.sort_values(by = 'IV', ascending = False).head(40)

<div class="alert alert-block alert-info">
    <span>
        <b>Conclusion: </b><br>IV is showing more fields impacted by a high bad rate. Crossing the first list and second one we can see these fields are linked together. It is decided to continue with first list.
    </span>    
</div>

<a id="T9"></a>
<h2>   
      <font size="6" , color = #26B260>
            10. Data preparation for modeling
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

<a id="T91"></a>
<h2>   
      <font size="5" , color = #41B6DF>
            10.1. Data preparation
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

In [ ]:
columns = ['CAT_AMT_GOODS_PRICE','CAT_DAYS_REGISTRATION','CAT_DAYS_ID_PUBLISH','CAT_REGION_RATING_CLIENT_W_CITY',
                'CAT_REG_CITY_NOT_WORK_CITY','CAT_EXT_SOURCE_2','CAT_EXT_SOURCE_3','CAT_DAYS_LAST_PHONE_CHANGE',
                'CAT_X_NAME_CONTRACT_TYPE','CAT_X_FLAG_OWN_CAR','CAT_X_NAME_EDUCATION_TYPE','CAT_X_NAME_HOUSING_TYPE',
                'CAT_NAME_FAMILY_STATUS_CHILD','CAT_NAME_INCOME_TYPE_DAYS_EMPLOYED','CAT_PHONE_EMAIL','TARGET']

columns_X=[c for c in columns]
columns_X.remove('TARGET')

df_X =df_application.loc[:,columns_X]
df_Y = df_application.loc[:,'TARGET']
df_XY =df_application.loc[:,columns]

In [ ]:
# Creation of disjunctive table
df_X = dummy_creation(df_X, columns_X)

In [ ]:
df_X.head()

In [ ]:
df_X=df_X.loc[:, df_X.columns.str.contains(':')]

<a id="T92"></a>
<h2>   
      <font size="5" , color = #41B6DF>
            10.2. Train/Validation tables
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

Creation of train and validation tables :

In [ ]:
from sklearn.model_selection import train_test_split

df_train_X, df_valid_X, df_train_Y, df_valid_Y = train_test_split(df_X, df_Y, test_size=0.3, random_state=0)
print(df_train_X.shape,"\n",df_valid_X.shape,"\n",df_train_Y.shape,"\n",df_valid_Y.shape)

In [ ]:
df_train_Y.value_counts(normalize=True) 

In [ ]:
df_valid_Y.value_counts(normalize=True) 

<a id="T10"></a>
<h2>   
      <font size="6" , color = #26B260>
            11. Score construction
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

<a id="T101"></a>
<h2>   
      <font size="5" , color = #41B6DF>
            11.1. Logistic regression
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

Model construction :

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# Define the gridSearch
reg = LogisticRegression(max_iter=1000, class_weight = 'balanced')
pipeline = Pipeline(steps=[('model', reg)])
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)

In [ ]:
# Build the score model
scores = cross_val_score(pipeline, df_train_X, df_train_Y, scoring = 'roc_auc', cv = cv)

# Show model performance based on train table
AUROC = np.mean(scores)
GINI = AUROC * 2 - 1
print('Mean AUROC: %.4f' % (AUROC))
print('Gini: %.4f' % (GINI))

# Complete calculation of the model on train table
pipeline.fit(df_train_X, df_train_Y)

Construction of the table with coefficients and fields names :

In [ ]:
# Column Feature names
summary_table = pd.DataFrame(columns = ['Feature name'], data = df_train_X.columns.values)
# Column Coefficients
summary_table['Coefficients'] = np.transpose(pipeline['model'].coef_)
# Correction of intercept
summary_table.loc[0] = ['Intercept', pipeline['model'].intercept_[0]]

In [ ]:
summary_table.loc[0:50,]

In [ ]:
summary_table.loc[51:100,]

<a id="T102"></a>
<h2>   
      <font size="5" , color = #41B6DF>
            11.2. Performance
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

In [ ]:
df_valid_X.shape

In [ ]:
from sklearn.metrics import roc_curve

# calculation on validation data
y_test = pipeline.predict(df_valid_X)
y_test_proba = pipeline.predict_proba(df_valid_X)

# ROC curve
fpr, tpr, thresholds = roc_curve(df_valid_Y, y_test_proba[:][: , 1])

plt.plot(fpr, tpr)
plt.plot(fpr, fpr, linestyle = '--', color = 'k')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve');

In [ ]:
from sklearn.metrics import roc_auc_score

# AUROC calculation
AUROC = roc_auc_score(df_valid_Y, y_test_proba[:][: , 1])
# Gini calculation
Gini = AUROC * 2 - 1
# print AUROC and Gini
print('AUROC: %.4f' % (AUROC))
print('Gini: %.4f' % (Gini))

In [ ]:
from sklearn.metrics import confusion_matrix
confmat = confusion_matrix(df_valid_Y, y_test)

<a id="T103"></a>
<h2>   
      <font size="5" , color = #41B6DF>
            11.3. Score grid
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

Construction of a score grid based on the logistic regression. The score will be between 0 and 500. 

In [ ]:
# Threshholds of scorecard
min_score = -1
max_score = 501

# Table creation from logistic regression coef
df_ref_categories = pd.DataFrame(summary_table.loc[1:100,"Feature name"].tolist(), columns = ['Feature name'])
df_scorecard =  summary_table.merge(df_ref_categories, on='Feature name', how='left').reset_index()

# Management of features
df_scorecard['Original feature name'] = df_scorecard['Feature name'].str.split(':').str[0]

# Min score observed
min_sum_coef = df_scorecard.groupby('Original feature name')['Coefficients'].min().sum()
# Max score observed
max_sum_coef = df_scorecard.groupby('Original feature name')['Coefficients'].max().sum()

In [ ]:
# Score points calculation to obtain 0-500 basis points
df_scorecard['Score - Calculation'] = df_scorecard['Coefficients'] * (max_score - min_score) / (max_sum_coef - min_sum_coef)
df_scorecard.loc[0, 'Score - Calculation'] = ((df_scorecard.loc[0,'Coefficients'] - min_sum_coef) /(max_sum_coef - min_sum_coef)) * (max_score - min_score) + min_score

# Round the score points
df_scorecard['Score - Final'] = df_scorecard['Score - Calculation'].round(0)

# Check min and max scores points
print(df_scorecard.groupby('Original feature name')['Score - Final'].min().sum())
print(df_scorecard.groupby('Original feature name')['Score - Final'].max().sum())

In [ ]:
df_scorecard[0:50]

<a id="T104"></a>
<h2>   
      <font size="5" , color = #41B6DF>
            11.4. Cutt offs
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

In [ ]:
# Calculate Youden's J-Statistic to identify the best threshhold
J = tpr - fpr
# locate the index of the largest J
ix = np.argmax(J)
best_thresh = thresholds[ix]
print('Best Threshold: %f' % (best_thresh))

In [ ]:
# Create a new dataframe for cutt offs
df_cutoffs = pd.DataFrame(thresholds, columns = ['thresholds'])

# Calculate the score corresponding to each threshold
df_cutoffs['Score'] = ((np.log(df_cutoffs['thresholds'] / (1 - df_cutoffs['thresholds'])) - 
                        min_sum_coef) * ((max_score - min_score) / (max_sum_coef - min_sum_coef)) + min_score).round()

In [ ]:
plt.hist(df_cutoffs['Score'])

In [ ]:
df_cutoffs['Score_Band'] = pd.qcut(df_cutoffs['Score'], 10, labels=[1,2,3,4,5,6,7,8,9,10])

In [ ]:
aggregations = {'thresholds': [sum,'count',"mean"],'Score': [min,max]}
df_cutoffs.groupby(["Score_Band"]).agg(aggregations).reset_index()

<a id="T105"></a>
<h2>   
      <font size="5" , color = #41B6DF>
            11.5. Score calculation on the whole database
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

Creation of the score table containing only score points to multiply with final dataframe :

In [ ]:
nomCol=df_scorecard.loc[:,["Feature name"]].values.tolist()
df_scorecard=df_scorecard.loc[:,["Score - Final"]]
df_scorecard_T=df_scorecard.reset_index(drop=True).transpose()
for col in range(len(nomCol)) : 
    type(nomCol[col])
    a=str(nomCol[col])
    a=a.replace("']", "")
    a=a.replace("['", "")
    df_scorecard_T.rename(columns={col:a}, inplace=True)

Multiplication with final dataframe :

In [ ]:
df_X_Score=df_X

for col in df_scorecard_T.columns.tolist() :
    if col!="Intercept": 
        dfcoef=df_scorecard_T[col].to_frame()
        coef=dfcoef.iloc[0,0]
        df_X_Score['Score_'+ col]=df_X_Score[col]*coef

In [ ]:
list = [col for col in df_X_Score.columns if col.startswith('Score_')]
df_X_Score['ScoreTot']=df_X_Score[list].sum(axis=1)    
df_X_Score['ScoreTot']=df_X_Score['ScoreTot']+df_scorecard_T["Intercept"].to_frame().iloc[0,0]

#df_X_Score.ScoreTot.loc[129565]

Score distribution : 

In [ ]:
plt.hist(df_X_Score['ScoreTot'])

Measure of default by score :

In [ ]:
df_X_Score = pd.concat([df_X_Score,df_Y], axis=1)
aggregations = {'TARGET': [sum,'count',"mean"]}
tab=df_X_Score.groupby(["ScoreTot"]).agg(aggregations).reset_index()
plt.plot(tab.iloc[:,0], tab.iloc[:,3])

<a id="T11"></a>
<h2>   
      <font size="6" , color = #26B260>
            12. Conclusion
        </font> 
    <div style="text-align:right"><a href='#Menu'>Menu</a></div>
</h2>

According to the fact the performance was showing good performance, we continued the scoring grid construction with the first model found. Performance could be better going deeper in model analysis of significant fields. </b><br>
To continue this analysis, it would be fine to use ML technics for features selection, grid search models, use stat model for the Logistic regression in order to analyze differencies with sklearn.

